In [1]:
from __future__ import division
import os

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
import tensorflow.compat.v1 as tf
#import tensorflow.contrib.slim as slim
import numpy as np

2022-11-10 04:05:45.932199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 04:05:46.108020: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 04:05:46.108057: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 04:05:46.145958: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 04:05:46.901025: W tensorflow/stream_executor/platform/de

In [ ]:
!pip install --upgrade tf_slim

In [4]:
import tf_slim as slim

In [5]:
!pip install rawpy

In [6]:
import rawpy
import glob

In [7]:
# !pip install scipy==1.2.0

In [8]:
import scipy as scipy

In [9]:
input_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/dataset/Fuji/Fuji/short/'
gt_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/dataset/Fuji/Fuji/long/'
checkpoint_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/ckpt4_Fuji/'
result_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/testresult4_dir/'

In [11]:
# get test IDs
test_fns = glob.glob(gt_dir + '1*.RAF') #For getting all the images with long exposure which start with 1 (i.e. Test images)
test_ids = [int(os.path.basename(test_fn)[0:5]) for test_fn in test_fns] #for storing the images IDs along with the test Tag(i.e. intial integer)

In [10]:
def lrelu(x):
    return tf.maximum(x * 0.2, x)


def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output


def network(input):  # Unet
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')

    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

    conv10 = slim.conv2d(conv9, 27, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    out = tf.depth_to_space(conv10, 3)
    return out


def pack_raw(raw):
    # pack X-Trans image to 9 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 1024, 0) / (16383 - 1024)  # subtract the black level

    img_shape = im.shape
    H = (img_shape[0] // 6) * 6
    W = (img_shape[1] // 6) * 6

    out = np.zeros((H // 3, W // 3, 9))

    # 0 R
    out[0::2, 0::2, 0] = im[0:H:6, 0:W:6]
    out[0::2, 1::2, 0] = im[0:H:6, 4:W:6]
    out[1::2, 0::2, 0] = im[3:H:6, 1:W:6]
    out[1::2, 1::2, 0] = im[3:H:6, 3:W:6]

    # 1 G
    out[0::2, 0::2, 1] = im[0:H:6, 2:W:6]
    out[0::2, 1::2, 1] = im[0:H:6, 5:W:6]
    out[1::2, 0::2, 1] = im[3:H:6, 2:W:6]
    out[1::2, 1::2, 1] = im[3:H:6, 5:W:6]

    # 1 B
    out[0::2, 0::2, 2] = im[0:H:6, 1:W:6]
    out[0::2, 1::2, 2] = im[0:H:6, 3:W:6]
    out[1::2, 0::2, 2] = im[3:H:6, 0:W:6]
    out[1::2, 1::2, 2] = im[3:H:6, 4:W:6]

    # 4 R
    out[0::2, 0::2, 3] = im[1:H:6, 2:W:6]
    out[0::2, 1::2, 3] = im[2:H:6, 5:W:6]
    out[1::2, 0::2, 3] = im[5:H:6, 2:W:6]
    out[1::2, 1::2, 3] = im[4:H:6, 5:W:6]

    # 5 B
    out[0::2, 0::2, 4] = im[2:H:6, 2:W:6]
    out[0::2, 1::2, 4] = im[1:H:6, 5:W:6]
    out[1::2, 0::2, 4] = im[4:H:6, 2:W:6]
    out[1::2, 1::2, 4] = im[5:H:6, 5:W:6]

    out[:, :, 5] = im[1:H:3, 0:W:3]
    out[:, :, 6] = im[1:H:3, 1:W:3]
    out[:, :, 7] = im[2:H:3, 0:W:3]
    out[:, :, 8] = im[2:H:3, 1:W:3]
    return out

In [11]:
np.random.seed(10)

In [12]:
import sys

In [13]:
import PIL
from PIL import Image

In [14]:
in_files = glob.glob(input_dir + '%05d_00*.RAF' % 10110)

In [15]:
in_files

[]

In [16]:
sess = tf.Session()

2022-11-10 04:06:47.568652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-10 04:06:47.568706: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-10 04:06:47.568745: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kopri): /proc/driver/nvidia/version does not exist
2022-11-10 04:06:47.569411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
tf.disable_eager_execution()
in_image = tf.placeholder(tf.float32, [None, None, None, 9])

In [20]:
print(in_image)

Tensor("Placeholder:0", shape=(None, None, None, 9), dtype=float32)


In [21]:
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])

In [22]:
out_image = network(in_image)

/Applications/Anaconda/anaconda3/envs/tf/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [23]:
print(out_image)

Tensor("DepthToSpace:0", shape=(None, None, None, 3), dtype=float32)


In [24]:
saver = tf.train.Saver()

In [25]:
sess.run(tf.global_variables_initializer())

2022-10-10 00:23:18.810286: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [26]:
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)

In [27]:
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)

loaded /Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/ckpt4_Fuji/model.ckpt
INFO:tensorflow:Restoring parameters from /Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/ckpt4_Fuji/model.ckpt


In [28]:
if not os.path.isdir(result_dir + 'final/'):
    os.makedirs(result_dir + 'final/')

In [29]:
for test_id in test_ids:
    # test the first image in each sequence
    in_files = glob.glob(input_dir + '%05d_00*.RAF' % test_id)
    print(len(in_files),end = ',')

2,2,2,2,2,3,2,2,3,2,2,2,3,2,2,2,3,2,2,2,3,3,2,3,3,2,3,3,2,2,2,2,2,2,2,3,2,2,2,3,2,

In [30]:
k=0
for test_id in test_ids:
    # test the first image in each sequence
    in_files = glob.glob(input_dir + '%05d_00*.RAF' % test_id)
    print(f'for Test ID = {test_id}: ')
    for k in range(len(in_files)): 
        try: 
            in_path = in_files[k] #For first image in kth sequence
            in_fn = os.path.basename(in_path)
            print(in_fn) #Printing the image name for first image in kth sequence
            gt_files = glob.glob(gt_dir + '%05d_00*.RAF' % test_id) #Accessing corresponding long exposure images
            gt_path = gt_files[0] #For taking path as string
            gt_fn = os.path.basename(gt_path) # long exposure Image file name
            in_exposure = float(in_fn[9:-5])
            gt_exposure = float(gt_fn[9:-5])
            ratio = min(gt_exposure / in_exposure, 300)

            raw = rawpy.imread(in_path) #Reading the first short exposure image
            input_full = np.expand_dims(pack_raw(raw), axis=0) * ratio
            im = raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
            # scale_full = np.expand_dims(np.float32(im/65535.0),axis = 0)*ratio #scale the low-light image using the same ratio
            scale_full = np.expand_dims(np.float32(im / 65535.0), axis=0)

            gt_raw = rawpy.imread(gt_path)
            im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
            gt_full = np.expand_dims(np.float32(im / 65535.0), axis=0)

            input_full = np.minimum(input_full, 1.0)

            output = sess.run(out_image, feed_dict={in_image: input_full})
            output = np.minimum(np.maximum(output, 0), 1)

            _, H, W, _ = output.shape

            output = output[0, :, :, :]
            gt_full = gt_full[0, 0:H, 0:W, :]
            scale_full = scale_full[0, 0:H, 0:W, :]
            scale_full = scale_full * np.mean(gt_full) / np.mean(
                scale_full)  # scale the low-light image to the same mean of the groundtruth

            Image.fromarray((output*255).astype(np.uint8),mode='RGB').save(
                result_dir + 'final/%5d_00_%d_out.png' % (test_id, ratio))
            Image.fromarray((scale_full*255).astype(np.uint8),mode='RGB').save(
                result_dir + 'final/%5d_00_%d_scale.png' % (test_id, ratio))
            Image.fromarray((gt_full*255).astype(np.uint8),mode='RGB').save(
                result_dir + 'final/%5d_00_%d_gt.png' % (test_id, ratio))
            #break
        except:
            print(f"Oops! {sys.exc_info()[0]} occurred for Test ID {test_id}, moving to next testing ID....")
    k+=1
    if k>=15:
        break

for Test ID = 10049: 
10049_00_0.033s.RAF
10049_00_0.1s.RAF
for Test ID = 10055: 
10055_00_0.033s.RAF
10055_00_0.1s.RAF
for Test ID = 10119: 
10119_00_0.04s.RAF
Oops! <class 'rawpy._rawpy.LibRawFileUnsupportedError'> occurred for Train ID 10119, moving to next training ID....
10119_00_0.1s.RAF
Oops! <class 'rawpy._rawpy.LibRawFileUnsupportedError'> occurred for Train ID 10119, moving to next training ID....
for Test ID = 10090: 
10090_00_0.04s.RAF
10090_00_0.1s.RAF
for Test ID = 10089: 
10089_00_0.1s.RAF
10089_00_0.04s.RAF
for Test ID = 10155: 
10155_00_0.1s.RAF
Oops! <class 'rawpy._rawpy.LibRawFileUnsupportedError'> occurred for Train ID 10155, moving to next training ID....
10155_00_0.033s.RAF
Oops! <class 'rawpy._rawpy.LibRawFileUnsupportedError'> occurred for Train ID 10155, moving to next training ID....
10155_00_0.04s.RAF
Oops! <class 'rawpy._rawpy.LibRawFileUnsupportedError'> occurred for Train ID 10155, moving to next training ID....
for Test ID = 10019: 
10019_00_0.1s.RAF
1001